In [ ]:
# Programming-Poetry-Project
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# File Location
Poetry_Analysis = "Resources/Poetry_Analysis_Data.csv"

# Read Poetry Analysis Data File As A Pandas Dataframe
Poetry_Analysis_df = pd.read_csv(Poetry_Analysis)
Poetry_Analysis_df.head(5)

In [ ]:
# Total Genre per Age – Stacked Column Chart (Bilal)
# Create a table with the Age and Genre Type
genre_per_age = Poetry_Analysis_df.groupby(['age', 'type'])['type'].count().unstack()
genre_per_age

In [ ]:
# Create a Stacked Column Chart with the table created with groupby from above
genre_per_age.plot(kind='bar', stacked=True, rot=20, alpha=0.8)
plt.title("Total Genre per Age", fontsize=16)
plt.xlabel("Age", fontsize=12)
plt.ylabel("Count of Genres", fontsize=12)
plt.legend(loc="upper left", title = "Genre", fontsize=9.6)
plt.ylim(0, 385)

# Save the graph to the images folder
plt.tight_layout()
plt.savefig("Images/genre_per_age.png")
plt.show()

In [ ]:
# Gender – Bar Graph (Bilal)

# Create a bar graph from the "Gender" column in the dataframe
Poetry_Analysis_df['Gender'].value_counts().plot(kind='bar', figsize=(7, 6), rot=20, color='b', alpha=0.6)
plt.xlabel("Gender", fontsize=12)
plt.ylabel("Count of People", fontsize=12)
plt.title("Gender – Bar Graph", fontsize=16)

# Save the graph to the images folder
plt.tight_layout()
plt.savefig("Images/gender.png")
plt.show()

row_count = print("Number of rows present:", 
      len(Poetry_Analysis_df['Gender']))
Poetry_Analysis_df['Gender'].value_counts()